# 11411/611 – NLP (S23)


## HW3 – Language Models

**File version:** 1.0.0

Whether for transcribing spoken utterances as correct word sequences or generating coherent human-like text, language models are extremely useful.

In this assignment, you will be building your own language model powered by n-grams.

### There are two major components in this HW:
#### Part 1: Programming [60 marks]
You are required to program an n-gram language model.

#### Part 2: Analyses [40 marks]
After writing the code, you are required to answer the empirical questions in the handout

### Submission Guidelines

**Deadline:** February 24th, 2023 at 11:59pm EST

**Programming:** 
- This notebook contains helpful test cases and additional information about the programming part of the HW. However, you are only required to submit `lm.py` and `utils.py` on Gradescope.
- Remember to upload the data.zip file provided as part of the handout
- We recommended that you first code in the notebook and then copy the corresponding methods/classes to `lm.py`.

**Written:**
- Analyses questions would require you to run your code.
- You need to write your answers in a document and upload it alongside the programming components


### Upload main.py and utils.py, and the data.zip file

In [ ]:
!unzip data.zip

## Part 1: Language Models [60 points]

### Step 0: Importing essential libraries

In [ ]:
from collections import Counter
from itertools import product
import math

### Step 1: Preprocessing

We provide you with a few functions in `utils.py` to read and preprocess your input data. Do not edit this file!

In [ ]:
from utils import *

We have performed a round of preprocessing on the datasets.

- Each file contains one sentence per line.
- All punctuation marks have been removed.
- Each line is a sequences of tokens separated by whitespace.

#### Special Symbols ( Already defined in `utils.py` )
The start and end tokens will act as padding to the given sentences, to make sure they are correctly defined, print them here:

In [ ]:
print("Sentence START symbol: {}".format(START))
print("Sentence END symbol: {}".format(EOS))


#### Reading and processing an example file

In [ ]:
# Read the sample file
sample = read_file("data/sample.txt")
print(sample)

In [ ]:
# Preprocess the content to add corresponding number of start and end tokens. Try out the method with n = 3 and n = 4 as well.
# Preprocessing example for bigrams (n=2)
sample = preprocess(sample, n=2)
print(sample)

### Step 2: Helper Functions

In [ ]:
# TODO
def flatten(lst):
    """
    Flattens a nested list into a 1D list.
    Args:
        lst: Nested list (2D)
    
    Returns:
        Flattened 1-D list
    """
    raise NotImplementedError

In [ ]:

assert flatten([["a", "b", "c"], ["d"]]) == ['a', 'b', 'c', 'd']

### Step 3: Get started!

### TO DO: `get_ngrams()`

In [ ]:
#######################################
# TO-DO: get_ngrams()
#######################################
def get_ngrams(list_of_words, n):
    """
    Returns a list of n-grams for a list of words.
    Args
    ----
    list_of_words: List[str]
        List of already preprocessed and flattened (1D) list of tokens e.g. ["<s>", "hello", "</s>", "<s>", "bye", "</s>"]
    n: int
        n-gram order e.g. 1, 2, 3
    
    Returns:
        n_grams: List[Tuple]
            Returns a list containing n-gram tuples
    """

    raise NotImplementedError

In [ ]:
sample = read_file("data/sample.txt")
sample = preprocess(sample, n=3)

assert get_ngrams(flatten(sample), 3) == [('<s>', '<s>', 'we'),
 ('<s>', 'we', 'are'),
 ('we', 'are', 'never'),
 ('are', 'never', 'ever'),
 ('never', 'ever', 'ever'),
 ('ever', 'ever', 'ever'),
 ('ever', 'ever', 'ever'),
 ('ever', 'ever', 'getting'),
 ('ever', 'getting', 'back'),
 ('getting', 'back', 'together'),
 ('back', 'together', '</s>'),
 ('together', '</s>', '<s>'),
 ('</s>', '<s>', '<s>'),
 ('<s>', '<s>', 'we'),
 ('<s>', 'we', 'are'),
 ('we', 'are', 'the'),
 ('are', 'the', 'ones'),
 ('the', 'ones', 'together'),
 ('ones', 'together', 'we'),
 ('together', 'we', 'are'),
 ('we', 'are', 'back'),
 ('are', 'back', '</s>')]

### **TO DO:** Class `LanguageModel()`

*Now*, we will define our LanguageModel class.

**Some Useful Variables:**
- self.model: `dict` of n-grams and their corresponding probabilities, key's being the tuple containing the n-gram, and the value being the probability of the n-gram.
- self.vocab: `dict` of unigram vocabulary with counts, key's being the words themselves and the values being their frequency.
- self.n: `int` value for n-gram order (e.g. 1,2,3).
- self.train_data: `List[List]` containing preprocessed **unflattened** train sentences. You will have to flatten it to use in the Language Model
- self.smoothing: `float` flag signifying the smoothing parameter (was called `k` in the previous HW).

In `lm.py`, we will be taking most of these argumemts from command line using this command:

`python3 lm.py --train data/sample.txt --test data/sample.txt --n 3 --smoothing 0 --min_freq 1`

Note that we will not be using Log probabilities in this section. Store the probabilities as they are, not in the log space.


## Laplace Smoothing

There's 2 ways to perform this:
- Either you calculate all possible n-gram at train time and calculate smooth probabilities for all of them, hence inflating the model (Eager Smoothing). You then use the probabilities as when required at test time. **OR**
- You calculate the probabilities for the **observed n-grams** at train time, using the smoothed likelihood formula, then if any unseen n-gram is observed at test time, you calculate the probability using the smoothed likelihood formula and store it in the model for future use. (Lazy Smoothing)

You will be implementing Lazy Smoothing

In [ ]:
#######################################
# TO-DO: LanguageModel()
#######################################
class LanguageModel():
    def __init__(self, n, train_data, alpha=1):
        """
        Language model class.
        
        Args
        ____
        n: int
            n-gram order
        train_data: List[List]
            already preprocessed unflattened list of sentences. e.g. [["<s>", "hello", "my", "</s>"], ["<s>", "hi", "there", "</s>"]]
        alpha: float
            Smoothing parameter
        
        Other attributes:
            self.tokens: list of individual tokens present in the training corpus
            self.vocab: vocabulary dict with counts
            self.model: n-gram language model, i.e., n-gram dict with probabilties
            self.n_grams_counts: dictionary for storing the frequency of ngrams in the training data, keys being the tuple of words(n-grams) and value being their frequency
            self.prefix_counts: dictionary for storing the frequency of the (n-1) grams in the data, similar to the self.n_grams_counts
            As an example:
            For a trigram model, the n-gram would be (w1,w2,w3), the corresponding [n-1] gram would be (w1,w2)
        """
        self.n = n
        self.train_data = train_data
        self.n_grams_counts = {}
        self.prefix_counts = {}
        self.alpha = alpha
        
        # Fill in the following two lines of code
        self.tokens = #TODO
        self.vocab  = #TODO

        self.model = self.build()


    def build(self):
        """
        Returns a n-gram dict with their smoothed probabilities. Remember to consider the edge case of n=1 as well
        
        You are expected to update the self.n_grams_counts and self.prefix_counts, and use those calculate the probabilities. 
        """
        # Extract n-grams from the flattened training data [update n_grams_counts]
        
        # Calculate the prefix (n-1 grams) count using the extracted n-grams [update prefix_counts]
        
        # Calculate probabilities using the get_smooth_probabilities function, you need to define the function

        # Return the probabilities
        
        raise NotImplementedError
    
    def get_smooth_probabilites(self,n_gram):
        """
        Returns the smoothed probability of the n-gram, using Laplace Smoothing. 
        Remember to consider the edge case of  n = 1
        HINT: Use self.n_gram_counts, self.tokens and self.prefix_counts 
        """
        
        raise NotImplementedError

In [ ]:
# Quick test
sample = read_file("data/sample.txt")
sample = preprocess(sample, n=2)
# For the sake of understanding we will pass alpha as 0 (No smoothing), so that you gain intuition about the probabilities
test_lm = LanguageModel(n=2, train_data=sample, alpha=0)

In [ ]:
assert test_lm.vocab == Counter({'<s>': 2,
         'we': 3,
         'are': 3,
         'never': 1,
         'ever': 4,
         'getting': 1,
         'back': 2,
         'together': 2,
         '</s>': 2,
         'the': 1,
         'ones': 1})

In [ ]:
assert test_lm.model =={('<s>', 'we'): 1.0,
 ('we', 'are'): 1.0,
 ('are', 'never'): 0.3333333333333333,
 ('never', 'ever'): 1.0,
 ('ever', 'ever'): 0.75,
 ('ever', 'getting'): 0.25,
 ('getting', 'back'): 1.0,
 ('back', 'together'): 0.5,
 ('together', '</s>'): 0.5,
 ('</s>', '<s>'): 1.0,
 ('are', 'the'): 0.3333333333333333,
 ('the', 'ones'): 1.0,
 ('ones', 'together'): 1.0,
 ('together', 'we'): 0.5,
 ('are', 'back'): 0.3333333333333333,
 ('back', '</s>'): 0.5}

Let's see what happens when we add smoothing

In [ ]:
sample = read_file("data/sample.txt")
sample = preprocess(sample, n=2)
test_lm = LanguageModel(n=2, train_data=sample, alpha=1)

In [ ]:
assert test_lm.vocab == Counter({'<s>': 2,
         'we': 3,
         'are': 3,
         'never': 1,
         'ever': 4,
         'getting': 1,
         'back': 2,
         'together': 2,
         '</s>': 2,
         'the': 1,
         'ones': 1})

In [ ]:
assert test_lm.model =={('<s>', 'we'): 0.23076923076923078,
 ('we', 'are'): 0.2857142857142857,
 ('are', 'never'): 0.14285714285714285,
 ('never', 'ever'): 0.16666666666666666,
 ('ever', 'ever'): 0.26666666666666666,
 ('ever', 'getting'): 0.13333333333333333,
 ('getting', 'back'): 0.16666666666666666,
 ('back', 'together'): 0.15384615384615385,
 ('together', '</s>'): 0.15384615384615385,
 ('</s>', '<s>'): 0.16666666666666666,
 ('are', 'the'): 0.14285714285714285,
 ('the', 'ones'): 0.16666666666666666,
 ('ones', 'together'): 0.16666666666666666,
 ('together', 'we'): 0.15384615384615385,
 ('are', 'back'): 0.14285714285714285,
 ('back', '</s>'): 0.15384615384615385}


Handling Unigram language model

In [ ]:
sample = read_file("data/sample.txt")
sample = preprocess(sample, n=1)
test_lm = LanguageModel(n=1, train_data=sample, alpha=1)

In [ ]:
assert test_lm.vocab == Counter({'<s>': 2,
         'we': 3,
         'are': 3,
         'never': 1,
         'ever': 4,
         'getting': 1,
         'back': 2,
         'together': 2,
         '</s>': 2,
         'the': 1,
         'ones': 1})

In [ ]:
assert test_lm.model == {('<s>',): 0.09090909090909091,
 ('we',): 0.12121212121212122,
 ('are',): 0.12121212121212122,
 ('never',): 0.06060606060606061,
 ('ever',): 0.15151515151515152,
 ('getting',): 0.06060606060606061,
 ('back',): 0.09090909090909091,
 ('together',): 0.09090909090909091,
 ('</s>',): 0.09090909090909091,
 ('the',): 0.06060606060606061,
 ('ones',): 0.06060606060606061}

### **TO DO:**  `perplexity()`

## **TODO**: Perplexity()
Steps:
1. Flatten the test data
2. Extract ngrams from the flattened data
3. Calculate perplexity according to given formula. For unseen n-gram, calculate using smoothed likelihood  and store the unseen n-gram probability in the labguage model `model` attribute

$PP(W_{test}) = PP(W_1W_2 ... W_n)^{-1/n} $

Tips:
- Remember that product changes summation under `log`. Take log of probabilities (`math.log()`), sum them up (`sum()`) and then exponentiate it (`math.exp()`) to get back to the original scale.
- Make sure to `flatten()` your data before creating the n_grams using `get_ngrams().`
- The test suite provided is **not exhaustive**

In [ ]:
#######################################
# TO-DO: perplexity()
#######################################
def perplexity(lm, test_data):
    """
    Returns perplexity calculated on the test data.
    Args
    ----------
    test_data: List[List] 
        Already preprocessed nested list of sentences
        
    lm: LanguageModel class object
        To be used for retrieving lm.model, lm.n and lm.vocab

    Returns
    -------
    float
        Calculated perplexity value
    """
    
    raise NotImplementedError

In [ ]:
# Quick test
test_lm = LanguageModel(n=2, train_data=sample, alpha=0)
test_ppl = perplexity(test_lm, sample)
assert test_ppl < 1.7
assert test_ppl > 0

In [ ]:
test_lm = LanguageModel(n=2, train_data=sample, alpha=1)
test_ppl = perplexity(test_lm, sample)
assert test_ppl < 5.0
assert test_ppl > 0

### Step 4: Bringing everything together!

**Note:** Most of these will already be defined for you in `main()` method in `lm.py`

In [ ]:
# Arguments

train_path = "data/bbc/tech.txt"
test_path = "data/bbc/tech.txt"
n = 3
min_freq = 1
smoothing = 0.1

In [ ]:

train = read_file(train_path)
test = read_file(test_path)

In [ ]:
print("No of sentences in train file: {}".format(len(train)))
print("No of sentences in test file: {}".format(len(test)))

In [ ]:
print("Raw train example: \n{}".format(train[2]))
print("Raw test example: \n{}".format(test[2]))

In [ ]:
# Basic preprocessing
train = preprocess(train, n)
test = preprocess(test, n)

In [ ]:
print("Preprocessed train example: \n{}\n".format(train[2]))
print("Preprocessed test example: \n{}".format(test[2]))

In [ ]:
print("Loading {}-gram model.".format(n))
lm = LanguageModel(n, train, smoothing)
print("Vocabulary size (unique unigrams): {}".format(len(lm.vocab)))
print("Total number of unique n-grams: {}".format(len(lm.model)))

assert len(lm.vocab) == 12133
assert len(lm.model) == 145967

In [ ]:
# Calculate perplexity

ppl = perplexity(lm, test)
print("Model perplexity: {:.3f}".format(ppl))
assert ppl <= 443.00

### Step 5: Analysis

**Note**: These methods are already written for you. Use them to solve Written subtasks

In [ ]:
import random

random.seed(11411)

In [ ]:
def best_candidate(lm, prev, i, without=[], mode="random"):
    """
    Returns the most probable word candidate after a given sentence.
    """
    blacklist  = ["<UNK>"] + without
    candidates = ((ngram[-1],prob) for ngram,prob in lm.model.items() if ngram[:-1]==prev)
    candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
    candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
    if len(candidates) == 0:
        return ("</s>", 1)
    else:
        if(mode=="random"):
            return candidates[random.randrange(len(candidates))]
        else:
            return candidates[0]

In [ ]:
def top_k_best_candidates(lm, prev, k, without=[]):
    """
    Returns the K most-probable word candidate after a given n-1 gram.
    Args
    ----
    lm: LanguageModel class object
    
    prev: n-1 gram
        List of tokens n
    """
    blacklist  = ["<UNK>"] + without
    candidates = ((ngram[-1],prob) for ngram,prob in lm.model.items() if ngram[:-1]==prev)
    candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
    candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
    if len(candidates) == 0:
        return ("</s>", 1)
    else:
        return candidates[:k]

In [ ]:
def generate_sentences_from_phrase(lm, num, sent, prob, mode):
    """
    Generate sentences using the trained language model.
    """
    min_len=12
    max_len=24
    
    for i in range(num):
        while sent[-1] != "</s>":
            prev = () if lm.n == 1 else tuple(sent[-(lm.n-1):])
            blacklist = sent + (["</s>"] if len(sent) < min_len else [])

            next_token, next_prob = best_candidate(lm, prev, i, without=blacklist, mode=mode)
            sent.append(next_token)
            prob *= next_prob
            
            if len(sent) >= max_len:
                sent.append("</s>")

        yield ' '.join(sent), -1/math.log(prob)

## Part 2: Written [40 points]. We have given some code for some of the written parts to make it easier for you.

### **Written 4.2** – Song Attribution [8 points]

In [ ]:
# Example code for Taylor Swift LM
n = 3
smoothing = 0.1
min_freq = 1

train = read_file("data/lyrics/taylor_swift.txt")
test = read_file("data/lyrics/test_lyrics.txt")

train = preprocess(train, n)
test = preprocess(test, n)
lm = LanguageModel(n, train, smoothing)

ppl = perplexity(lm, test)
print(ppl)

### **Written 4.3.1** –  Intro to Decoding [8 points]

In [ ]:
n = 3
smoothing = 0.1
min_freq = 1

In [ ]:
train = read_file("data/bbc/entertainment.txt")
train = preprocess(train, n)
lm = LanguageModel(n, train, smoothing)

In [ ]:
s1 = ("number", "three")

s2 = ("starred", "in")

s3 = ("actor", "in")

In [ ]:
print(top_k_best_candidates(lm, s1, 5, without=['<s>', '</s>']))

### **Written 4.3.2** – Text Generation [8 points]

For this subtask, use the LM trained in Written 3.4.1

Selecting words sequentially from a probability distribution is called _decoding_.

Two popular decoding approaches are,
1. **Max-probability decoding** - We consistently choose the candidate with maximum probability.
2. **Random Sampling** - We sample a candidate randomly.
2. **top-K Sampling** - We sample a candidate randomly from the top-K most probable choices.

In this part, we will try the first two approaches to generate sentences.

Q1. Use `generate_sentences()` method to generate sentences after the provided phrases from `s1` to `s3`. Use modes `random` and `max`. Report one of your favourite generations (for any strategy or phrase) along with its probability score.

Q2. Which decoding strategy did you like better and why?

In [ ]:
# Random
for _ in range(5):
    print(list(generate_sentences_from_phrase(lm, 1, ["<s>", "<s>", "number", "three"], 0.2, mode="random")))

**Aside (for fun!)**: Train your LM on Taylor Swift lyrics and generate the next hit!

### **Written 4.4** – Battle of the LMs: GPT-2 vs Trigram [8 points]

For this subtask, you will be generating text and comparing GPT-2 with your n-gram language model. 

Generative pretrained transformer (GPT) is a neural language model series created by OpenAI. The n-gram language model you trained has on average around 10K-20K parameters (`len(lm.model)`.) Compare that to the 175 billion parameters of the latest version of GPT-3!

Let's see how GPT-2 compares to the LM you trained in Written 4.3.1 using `data/bbc/tech-small.txt` as the test dataset.

In [ ]:
# Calculate your n-gram model's perplexity
test = read_file("data/bbc/tech-small.txt")
test = preprocess(test,3)

perplexity(lm,test)

### Computing GPT-2's perplexity on test set

You need to enable a GPU runtime from `Runtime` menu option. Go to `Runtime` → `Change Runtime Type` → `Hardware Accelerator (GPU)`

In [ ]:
!pip install transformers
!pip install torch
!pip install tqdm

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch

model_id = "distilgpt2"
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [ ]:
test = read_file("data/bbc/tech-small.txt")
encodings = tokenizer("\n\n".join(test), return_tensors="pt")

In [ ]:
from tqdm import tqdm

max_length = model.config.n_positions
stride = 100

nlls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0] * trg_len

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)

In [ ]:
print("Perplexity using GPT2:", ppl.item())